In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from IPython.display import display

x_train = pd.read_csv("data/fashion-mnist_train.csv")
x_test = pd.read_csv("data/fashion-mnist_test.csv")

y_train_labels = x_train['label']
y_test_labels = x_test['label']

x_train.drop(["label"], axis=1, inplace=True)
x_test.drop(["label"], axis=1, inplace=True)


img_array = np.array(x_train.iloc[3].values, dtype=np.uint8)
img_reshape = img_array.reshape((28,28))
image = Image.fromarray(img_reshape)
display(image)

2024-07-08 23:11:09.203124: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 23:11:09.220838: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 23:11:09.356843: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 23:11:09.528177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 23:11:09.691629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

## Data preparation

In [2]:
# Label encoding
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train_labels.to_frame())
y_test = encoder.fit_transform(y_test_labels.to_frame())

# Data normalization
x_train = (x_train / 255) - 0.5
x_test  = (x_test / 255) - 0.5

'''
28, 28, 1 specifies the dimensions of each image: height, width, and channels (1 for grayscale).

-1 in the reshape() function automatically calculates the batch size based on the total number of samples in x_train
'''

# Reshape the image
x_train = x_train.values.reshape(-1, 28, 28, 1) 
x_test = x_test.values.reshape(-1, 28, 28, 1) 

# x_test = np.expand_dims(x_test, axis=-3)

## Train

In [4]:
conv_num_filters = 8
conv_filter_size = 3
pool_size = 2

print(x_train.shape)
model = Sequential([
    Conv2D(conv_num_filters,conv_filter_size),
    Conv2D(conv_num_filters,conv_filter_size),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(12, activation='relu'),
    Dense(12, activation='relu'),
    Dense(10, activation='softmax') # 10 neurons = 10 oufits
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(
    x_train,
    y_train,
    epochs=6,
    validation_data=(x_test,y_test),
    
)

(60000, 28, 28, 1)
Epoch 1/6
1317/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6664 - loss: 0.9121